# Basic instructions 

Download the '01. Outdoor-Indoor' past and copy paste it localization on the variavel seach_pattern_for_CSV

# Bibliotecas usadas

In [1]:
import pandas as pd
import glob
import re
from datetime import datetime, timedelta
import numpy as np

# Bases "fisiologicas"

In [3]:
padrao_busca_csv = "/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments/**/*.csv"
todos_arquivos = glob.glob(padrao_busca_csv, recursive=True)

ids = ["IS", "LG", "LM", "MF", "TM", "ZM", "AW", "JF", "JP", "MN", "NC", "RM"] 
nao = ["01. Pilot LabEEE 20231114", "02. Pilot LabEEE 20231122"]

arquivos_fisiologicos = [arquivo for arquivo in todos_arquivos if any(sigla in arquivo for sigla in ids) and not any(nome in arquivo for nome in nao)]

In [4]:
relacao_tabela_informacao = {"A (°C)" :	["4B00000069A8E941","FE00000069AF1041","B600000069B23241","7D00000069AB4641","0F00000069AAAF41","C400000069ADCC41"],
                            "D (°C)":	["CB00000069AD3041","9400000069AA4941","2700000069AD8541","C100000069B26141","CA00000069B19041","4A00000069AC0841"],
                            "F (°C)":	["C400000069AA8341","C400000069B3DB41","8100000069AE1641","2700000069AACA41","6700000069AEFC41","1C00000069B3F941"], 
                            "H (°C)":	["B200000069AE3141","EC00000069A9CD41","3300000069AC2241","3600000069AAD741","E800000069AAC041","AD00000069AF4C41"],
                            "J (°C)":	["FA00000069AD0441","3400000069B1AE41","C700000069B34D41","4B00000069B0A841","0600000069B22741","9100000069AAEA41","6700000069AB8141"],
                            "K (°C)":	["FB00000069B28F41","7B00000069AA6A41","E500000069AA2F41","1A00000069A9BF41","FE00000069B10741","E100000069AD4641"],
                            "M (°C)":	["1B00000069B06241","8C00000069ADD241","F000000069AF6941","AA00000069AA8141","1600000069ABE741","A000000069AEBA41"],
                            "O (°C)":	["7600000069AA8541","F200000069B16D41","3000000069A9C941","BF00000069AFC741","A300000069B12241","D900000069ACA241"],
                            "Q (°C)":	["8600000069B0B141","6000000069AB3141","DB00000069A9CC41","EF00000069B23141","6900000069AE8541","4300000069B3CF41"],
                            "T (°C)":	["7600000069AB9C41","4500000069B1C841","6600000069B25C41","A200000069B0BE41","D700000069B3D541","9800000069B26241","9E00000069AE7241"],
                            "T_RH" : ["6E0000006CB57E41",	"590000006CB46641",	"4B0000006CBA7341",	"D30000006CBD4C41",	"370000006341C241",	"9C000000633F9F41"]}

nomes = ["Zac", "Joao", "Igor", "Luis", "Milena", "Ana", "Marcela", "Nathalia", "Thalita", "Liege", "Rayner", "João"]
sports = ["WALKING","CYCLING","CROSS-COUNTRY_SKIING","SKATING","RIDING","ROWING"]

ids_com_codigo = {"IS":'IS26', "LG":'LG37', "LM":'LM27', "MF":'MF36', "TM":'TM35', "ZM":'ZM26', "AW":'AW35', "JF":'JF25', "JP":'JP27', "MN":"MN37", "NC":"NC36", "RM":"RM26"}

In [5]:
def verifica_qual_tipo_de_csv(titulo):
    for chave, valores in relacao_tabela_informacao.items():
        if titulo in valores:
            return chave

def convertepfloat(row):
    n1 = str(row.iloc[2]).replace('.', '')
    n2 = str(row.iloc[3]).replace('.', '')
    return float(n1 + '.' + n2)

def datamento(base_de_dados_crua, base_de_dados_nova):
    dia = base_de_dados_crua.iloc[0,2]
    dia = dia.replace('-','/')
    hora_inicio = base_de_dados_crua.iloc[0,3]
    data_inicio = dia + " " + hora_inicio
    data_inicio = datetime.strptime(data_inicio, '%d/%m/%Y %H:%M:%S')
    data_fim = dia + ' ' + '20:00:00'
    data_fim = datetime.strptime(data_fim, '%d/%m/%Y %H:%M:%S')
    intervalos_tempo = pd.date_range(start=data_inicio, end=data_fim, freq='s')
    base_de_dados_nova['Date'] = intervalos_tempo[:len(base_de_dados_nova)]

def limpeza_simples(id, arquivo, coluna):
    colunas = ["Date", "uni", "u", "d"]
    dados = pd.read_csv(arquivo, skiprows=20,names=colunas).fillna('0')
    dados[f'{coluna}'] = dados.apply(lambda row: convertepfloat(row), axis=1)
    dados = dados.iloc[:, [0,-1]]
    dados['Date'] = pd.to_datetime(dados['Date'], format='%d/%m/%y %H:%M:%S')
    return dados

def calcular_TPond(row):
    if all(row[['A (°C)', 'D (°C)', 'F (°C)', 'H (°C)', 'J (°C)', 'K (°C)', 'M (°C)', 'O (°C)', 'Q (°C)', 'T (°C)']].notnull()):
        return float(0.07 * row['T (°C)'] + 0.13 * row['Q (°C)'] + 0.19 * row['O (°C)'] + 0.12 * row['M (°C)'] + 0.12 * row['K (°C)'] + 0.12 * row['J (°C)'] + 0.05 * row['H (°C)'] + 0.06 * row['F (°C)'] + 0.08 * row['D (°C)'] + 0.06 * row['A (°C)'])
    else:
        return None

def atribuir_id(id, df):
    for chave, valores in ids_com_codigo.items():
        if id in valores:
            df['id'] = chave

In [6]:
col_names = ['id', 'Date', 'TPond (°C)', 'A (°C)', 'D (°C)', 'F (°C)', 'H (°C)', 'J (°C)', 'K (°C)', 'M (°C)', 'O (°C)', 'Q (°C)',	'T (°C)', 'wHR (bpm)', 'cHR (bpm)', 'wU (bpm)', 'wT (°C)']
BaseDeDados = pd.DataFrame(columns=col_names)

In [7]:
for arquivo in arquivos_fisiologicos:
    ultimo_backslash = arquivo.rfind('\\')
    ultimounderscore = arquivo.rfind('_')
    titulo = arquivo[ultimo_backslash+1:ultimounderscore]
    penultimo_backslash = arquivo.rfind('\\', 0, ultimo_backslash)
    id = arquivo[penultimo_backslash+1:penultimo_backslash+3]
    if "Test+Room" in titulo or any(name in titulo for name in nomes):
        heart = pd.read_csv(arquivo).fillna('0')
        if(heart.iloc[0,1] == "OTHER_OUTDOOR"):
            pulso = pd.read_csv(arquivo, usecols=["HR (bpm)"], skiprows=2)
            datamento(heart,pulso)
            atribuir_id(id,pulso)
            pulso = pulso.rename(columns={'HR (bpm)': 'wHR (bpm)'})
            BaseDeDados = pd.concat([BaseDeDados, pulso], ignore_index=True)
        elif(heart.iloc[0,1] in sports):
            peito = pd.read_csv(arquivo, usecols=["HR (bpm)"], skiprows=2)
            datamento(heart,peito)
            atribuir_id(id,peito)
            peito = peito.rename(columns={'HR (bpm)': 'cHR (bpm)'})
            BaseDeDados = pd.concat([BaseDeDados, peito], ignore_index=True)
        elif(heart.iloc[0,1] == "False" or "True"):
            continue
        else:
            print(heart.iloc[0,1])    
    elif("_U" in arquivo) or ("_T" in arquivo) or (any(titulo in valores for valores in relacao_tabela_informacao.values())):
        if("_T" in arquivo):
            coluna = 'wT (°C)'
            temp = limpeza_simples(id, arquivo, coluna)
            atribuir_id(id,temp)
            BaseDeDados = pd.concat([BaseDeDados, temp],ignore_index=True)
        elif("_U" in arquivo):
            coluna = 'wU (bpm)'
            umid = limpeza_simples(id, arquivo, coluna)
            atribuir_id(id,umid)
            BaseDeDados = pd.concat([BaseDeDados, umid],ignore_index=True)
        else:
            coluna = verifica_qual_tipo_de_csv(titulo)
            temp = limpeza_simples(id, arquivo, coluna)
            atribuir_id(id,temp)
            BaseDeDados = pd.concat([BaseDeDados, temp],ignore_index=True)
    else:
        print(titulo)

In [8]:
BaseDeDados = BaseDeDados.groupby(['id', 'Date']).agg('mean').reset_index()

In [9]:
BaseDeDados['TPond (°C)'] = BaseDeDados.apply(calcular_TPond, axis=1)

In [10]:
BaseDeDados.to_csv('BaseDeDados.csv', index=False, encoding='utf-8')

# Ambientais + "fisiologicas"

In [11]:
siglas_ambientais = ['CSLAB','CSSR','HALL','OUT','TtSR']
arquivos_CSLAB = [arquivo for arquivo in todos_arquivos if 'CSLAB' in arquivo] 
arquivos_OUT = [arquivo for arquivo in todos_arquivos if 'OUT' in arquivo]
arquivos_CSSR = [arquivo for arquivo in todos_arquivos if 'CSSR' in arquivo]
arquivos_HALL = [arquivo for arquivo in todos_arquivos if 'HALL' in arquivo and 'excel' not in arquivo]
arquivos_TtSR = [arquivo for arquivo in todos_arquivos if 'TtSR' in arquivo] 

In [12]:
BaseDeDados_ambientais = pd.DataFrame()

In [13]:
def rename_cols(df, tipo):
    df = df.rename(columns={'Data': 'Date'})
    for coluna in df.columns[1:]:
        if('Unnamed' in coluna):
            df = df.drop(columns=[coluna])
            continue
        df = df.rename(columns={coluna: f"{coluna} {tipo}"})
    return df

In [14]:
for arquivo in arquivos_CSSR:
    try:
        importado = pd.read_csv(arquivo, sep=';')
        importado = rename_cols(importado, 'CSSR')
        importado = importado.iloc[:, [0,1,2,3,4,5]]
        importado['Date'] = pd.to_datetime(importado['Date'], format='%d/%m/%Y %H:%M:%S')
        BaseDeDados_ambientais = pd.concat([BaseDeDados_ambientais, importado], ignore_index=True)
    except:
        print(arquivo)

for i,arquivo in enumerate(arquivos_OUT):
    try:
        importado = pd.read_csv(arquivo, sep=';', usecols=['Data','T_Globo','Direcao','Veloc'])
        importado = rename_cols(importado, 'OUT')
        importado = importado.iloc[10:]
        if(i==4) or (i == 5):
            importado['Date'] = importado['Date'].apply(lambda x: x + ':00')
        importado['Date'] = pd.to_datetime(importado['Date'], format='%d/%m/%Y %H:%M:%S')
        BaseDeDados_ambientais = pd.concat([BaseDeDados_ambientais, importado], ignore_index=True)
    except:
        print(arquivo)

for i, arquivo in enumerate(arquivos_CSLAB):
    try:
        importado = pd.read_csv(arquivo, sep=';')
        if(i==5):
            importado = pd.read_csv(arquivo, sep=',')
        importado = rename_cols(importado, 'CSLAB')
        importado['Date'] = importado['Date'].apply(lambda x: x + ':00')
        importado['Date'] = pd.to_datetime(importado['Date'], format='%d/%m/%Y %H:%M:%S')
        BaseDeDados_ambientais = pd.concat([BaseDeDados_ambientais, importado], ignore_index=True)  
    except:
        print(arquivo)

arquivos_TtSR.pop(0)
for arquivo in arquivos_TtSR:
    TtSR = pd.read_csv(arquivo,encoding='latin1',delimiter=';',skiprows=1,skipfooter=40,engine='python') 
    TtSR = TtSR.iloc[:, [0,2,4,5]]
    TtSR = TtSR.rename(columns={'Data/Hora':'Date','094 [m/s]':'V_ar [m/s] TtSR','491 [Â°C]':'T_ar [°C] TtSR','517 TC1 [Â°C]':'T_globo [°C] TtSR'})
    TtSR['Date'] = pd.to_datetime(TtSR['Date'], format='%d/%m/%Y %H:%M:%S')
    BaseDeDados_ambientais = pd.concat([BaseDeDados_ambientais, TtSR], ignore_index=True)

colunas_usadas = ['Date-Time (Brazil Standard Time)','Ch:1 - Temperature   (°C)','Ch:2 - RH   (%)']
for arquivo in arquivos_HALL:
    HALL = pd.read_csv(arquivo,encoding='latin1',delimiter=';', usecols=colunas_usadas)
    HALL = HALL.rename(columns={'Date-Time (Brazil Standard Time)':'Date','Ch:1 - Temperature   (°C)':'Temp °C HALL','Ch:2 - RH   (%)':'RH % HALL'})
    HALL['Date'] = pd.to_datetime(HALL['Date'], format='%m/%d/%Y %H:%M:%S')
    BaseDeDados_ambientais = pd.concat([BaseDeDados_ambientais, HALL], ignore_index=True)

In [15]:
BaseDeDados_ambientais.to_csv('BaseDeDados ambientais.csv', index=False, encoding='utf-8')

In [16]:
BaseDeDados_final = pd.merge(BaseDeDados, BaseDeDados_ambientais, how = 'outer')

# Forms + "ambientais/fisiologicos"

In [18]:
Dados_qualitativos = pd.DataFrame()

In [22]:
arquivos_qualitativos  = [arquivo for arquivo in todos_arquivos if 'Probral' in arquivo and not 'Pilot' in arquivo]

9


In [ ]:
for arquivo in arquivos_qualitativos:
    A = pd.read_csv(arquivo, encoding='latin-1',sep=',')
    A.rename(columns={'Carimbo de data/hora':'Date'}, inplace=True)
    print(A.head())
    A['Date'] = pd.to_datetime(A['Date'], format='%d/%m/%Y %H:%M:%S')

In [ ]:
Dados_qualitativos.to_csv('Dados_qualitativos.csv', index=False, encoding='utf-8')
BaseDeDados_final = pd.merge(BaseDeDados_final, Dados_qualitativos, how = 'outer')
BaseDeDados_final.to_csv('BaseDeDados_final.csv', index=False, encoding='utf-8')